In [1]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/?/*
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 0
sim = 0
new = 1

Min_Entries = 100

In [3]:
isigma = 2
N_files = 9
N_layers = 16
N_hists = 4
N_phi = 6
N_ilyaers = 1
is_e = 1
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
phi_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
phi_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if new: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
if is_sigmalised: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/NewHadron/rungroup/"
file_names="my-12th_QA_19423_1078runs_rg" #my-25th_QA_19197_1000runs
if not new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new8_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [4]:
if sim :
    file_path="../sim/output/"
    file_names="my-kek"

    #phi_bins = [96,97,98,99,100,100,101,101,102,103,104,105]

In [5]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if new: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    else: infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_emc_read0 = []
    for ilayer in range(N_layers):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg0.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg1.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg2.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg3.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg4.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg5.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg6.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg7.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg8.root


In [6]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.5,0.8,1.2,1.6,2.0,2.4,3.0,3.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.6499999999999999, 0.75, 0.8500000000000001, 0.95, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.4, 2.0, 2.8, 4.9])


In [7]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]
arms = N_layers/8

In [8]:
def emc_fitter(InHist, iphi, seed=1, ilayer=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*ilayer*100
    iilayer = int(ilayer/arms)
    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    central_bean_prev = (0.001*phi_bins[2*iphi]-0.0005-0.1)
    if new: central_bean_prev = (0.002*phi_bins[2*iphi]-0.001-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(ilayer)+"/"+str(iphi)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{ilayer}_{iphi}",phi_bins[2*iphi],phi_bins[2*iphi],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        if emc_var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = 0.5* (pt_boarders[p]+pt_boarders[p+1])
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.

        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit[int(ilayer%2)][0]+central_bean_prev, boards_fit[int(ilayer%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{ilayer}_{iphi}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{ilayer}_{iphi}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(ilayer%2)][0], boards_tot[int(ilayer%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        g_gaus.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
        g_gaus2.SetParameter(1,0.5*(boards_fit2[iilayer][0]+boards_fit2[iilayer][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[iilayer][1]-boards_fit2[iilayer][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(ilayer%2)][1]/100,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if iphi>-1:
            total.SetParLimits(1,boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        if iOption0<4 and is_sigmalised==0:
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))
        else:
            Mean_local_err.append(total.GetParError(1)+ (0.15-0.005*p)*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+(0.15-0.005*p)*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]


In [9]:
def emc_centr_fitter(ilayer=1,InHist=hists_emc_read, phibins=[0,1,2,3,4,5], sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for iFile in range(N_files):
        out_hists1 = []
        for iphi in phibins:
            out_hists1.append(emc_fitter(InHist[iFile][ilayer][iOption0+is_sigmalised*2], iphi, iFile, ilayer, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [10]:
#Ntr = N_files
Ntr = N_layers
pool = mp.Pool(Ntr)
#output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
with pool:
    output_array = pool.map(emc_centr_fitter, [i for i in range(Ntr)])
pool.close()

In [11]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(N_files):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layers):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for iphi in range(N_phi):
            Means0.append(output_array[ilayer][ifile][iphi][0])
            Mean_errs0.append(output_array[ilayer][ifile][iphi][1])
            Sigmas0.append(output_array[ilayer][ifile][iphi][2])
            Sigma_errs0.append(output_array[ilayer][ifile][iphi][3])
            Int_FG0.append(output_array[ilayer][ifile][iphi][4])
            Int_BG0.append(output_array[ilayer][ifile][iphi][5])
            pt_arr0.append(output_array[ilayer][ifile][iphi][6])
            pt_arr_err0.append(output_array[ilayer][ifile][iphi][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [12]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.5)
                fit_functions1[-1].SetParLimits(1,0.001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [13]:
g_sigma = []
for ifile in range(N_files):
    g_sigma_f = []
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        g_sigma_layer = []
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_sigma_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Sigmas[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Sigma_errs[ifile][ilayer][iphi]))
            g_sigma_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_sigma_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if iOption0 == 0 and ilayer%2==0 and (iphi == 0 or iphi == N_phi-1): 
                g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","",0.2,1.2)
            else: g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","")
            g_sigma_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_sigma_f.append(g_sigma_layer)
    g_sigma.append(g_sigma_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

In [14]:
print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_ilayer in fit_functions_ifile:
        print("{")
        for fits in fit_functions_ilayer:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[9][16][6][3]={
{
{
{0.001606, 0.005835, -4.0786},
{0.001704, 0.007129, -4.499995},
{0.001588, 0.006427, -4.195251},
{0.001588, 0.006621, -4.5},
{0.001734, 0.005669, -4.5},
{0.00152, 0.007445, -4.5},
},
{
{0.001842, 0.007901, -4.5},
{0.001819, 0.005865, -4.5},
{0.001846, 0.005915, -4.499999},
{0.001922, 0.005781, -4.499999},
{0.001731, 0.007718, -4.499999},
{0.001813, 0.006411, -4.5},
},
{
{0.001744, 0.005936, -4.0},
{0.001709, 0.005714, -3.999903},
{0.001573, 0.006211, -3.6673},
{0.001611, 0.006935, -3.999968},
{0.001658, 0.006478, -3.999999},
{0.001754, 0.005044, -3.999997},
},
{
{0.001985, 0.005413, -3.999999},
{0.001981, 0.004992, -4.0},
{0.001834, 0.005041, -3.625305},
{0.00185, 0.004432, -3.498566},
{0.001957, 0.004771, -4.0},
{0.002, 0.004023, -3.780654},
},
{
{0.002, 0.01, -2.872052},
{0.002, 0.01, -2.853873},
{0.001731, 0.01, -2.667269},
{0.001422, 0.009157, -2.5},
{0.002, 0.01, -3.458605},
{0.001957, 0.01, -3.418228},
},
{
{0.002, 0.01, -2.843201},
{0.001725, 0

In [15]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0==0:
                    fit_functions1[-1].SetParLimits(2,-5.126,-0.23)
                    if ilayer%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-2.0,-1)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,0.1,1.3)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,1,2.0)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-1.3,-0.1)
                    if iphi==3 and ilayer ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        fit_functions1[-1].SetParameter(1,1.6)
                        fit_functions1[-1].SetParameter(2,-0.9)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [16]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    g_mean = [[[[]]*N_phi]*N_layers]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_mean[ifile][ilayer][iphi].append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Means[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Mean_errs[ifile][ilayer][iphi]))
            
            Format_Graph(g_mean[ifile][ilayer][iphi][-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            g_mean[ifile][ilayer][iphi][-1].Draw('P')
            if not is_sigmalised: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.7,1)
            else: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.5,1)
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][ilayer][iphi][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [17]:
if not is_sigmalised:
    print(f"mean_pt_params[{N_files}][{N_layers}][{N_phi}]="+"{")
    for fit_functions_file in fit_mean_functions:
        print("{")
        for fit_functions_syst in fit_functions_file:
            print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
        print("},")
    print("};")
else:
    print("const float "+iOptames[iOption0]+f"_sMean_pt_params[{N_files}][{N_layers}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        iilayer = 0
        for fits in fit_functions_ifile:
            print("{"+f"{round(fits[3-int(iilayer%2)].GetParameter(0),6)}, {round(fits[3-int(iilayer%2)].GetParameter(1),6)}, {round(fits[3-int(iilayer%2)].GetParameter(2),6)}"+"},")
            iilayer+=1
        print("},")      
    print("};")        
    print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        for fit_functions_ilayer in fit_functions_ifile:
            print("{")
            for fits in fit_functions_ilayer:
                print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
            print("},") 
        print("},")      
    print("};")        

mean_pt_params[9][16][6]={
{
{-0.012059, -0.009953,-0.00591, 0.000364, 0.008541, 0.010635},
{-0.012158, -0.010079,-0.005969, 8.3e-05, 0.008504, 0.010482},
{-0.010778, -0.008696,-0.00454, 0.001713, 0.009879, 0.01202},
{-0.01043, -0.008472,-0.004356, 0.001878, 0.010175, 0.012193},
{-0.013337, -0.011088,-0.006822, -0.000267, 0.008009, 0.01009},
{-0.013185, -0.010885,-0.006757, -0.000292, 0.008224, 0.010171},
{-0.010305, -0.008219,-0.003939, 0.002504, 0.011114, 0.01327},
{-0.010202, -0.00828,-0.003994, 0.002386, 0.010836, 0.012994},
{-0.014639, -0.01279,-0.008099, -0.00101, 0.008011, 0.010391},
{-0.014457, -0.012205,-0.00754, -0.000602, 0.008362, 0.010562},
{-0.010649, -0.008735,-0.00393, 0.003113, 0.012764, 0.015025},
{-0.010647, -0.00842,-0.003916, 0.003077, 0.012167, 0.014361},
{-0.014394, -0.01195,-0.006739, 0.000213, 0.009105, 0.01137},
{-0.014326, -0.011875,-0.006912, 0.000213, 0.008898, 0.011086},
{-0.011407, -0.009125,-0.004742, 0.002077, 0.012267, 0.014811},
{-0.011095, -0.008901,

In [18]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for iphi in range(N_phi):
    if not is_sigmalised and not new: dphi.append( round(0.001*phi_bins[2*iphi]-0.0005-0.1,5))
    if new and not is_sigmalised: dphi.append( round(0.002*phi_bins[2*iphi]-0.001-0.1,5))
    if is_sigmalised: dphi.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
    dphi_err.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParameter(0))
            dynamic_dphi_err_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-0.011, -0.009, -0.005, 0.001, 0.009, 0.011])
[[array('d', [-0.012059231503843953, -0.00995271471639438, -0.005910041043828609, 0.00036390258951004423, 0.008541487491744843, 0.010634805777318516]), array('d', [-0.012158101493310914, -0.010079162498676655, -0.0059691955105137355, 8.31760383897262e-05, 0.008504241783735968, 0.010482108703329954]), array('d', [-0.010778310987082946, -0.008695818040702973, -0.004540387022232317, 0.0017133688406360953, 0.009879349625378321, 0.012019525209204414]), array('d', [-0.010430316730709214, -0.008471794172619219, -0.004356077377093485, 0.001878212661145684, 0.010175247619955068, 0.012192855324095395]), array('d', [-0.013336721975601842, -0.011087823576103852, -0.006821927674054897, -0.00026748085357142584, 0.008008643517969125, 0.010090177006066174]), array('d', [-0.013184612256785983, -0.010884574651965, -0.006756515946709719, -0.0002918676631095754, 0.008223936836592425, 0.01017084599957089]), array('d', [-0.010304540163274884,

In [19]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 2, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "mean [rad]", 1.1, 1.1, 0.06, 0.05, "",0.2,0.2,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, dynamic_dphi[ifile][ilayer],dphi_err,dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_mean_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_mean_phi[ifile][ilayer][iphi][-1].Fit("pol1","Q","")
            g_mean_phi[ifile][ilayer][iphi][-1].GetFunction("pol1").SetLineColor(colors[ilayer%4])
            g_mean_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_phi_{ifile}{iOption0}{is_sigmalised}.png")

In [20]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_functions[ifile][ilayer][iphi].Eval(pt_point)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
            dynamic_dphi_err_ilayer.append((fit_functions[ifile][ilayer][iphi].GetParError(0)*2)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [1.0146206392184811, 1.05316271008708, 1.0194204127283264, 0.9805795872716735, 0.9999313533112187, 0.98865272416462]), array('d', [1.0698992493281154, 0.9749289396366677, 0.9876522616251853, 1.012347738374815, 1.0177151297072198, 0.9952765368256719]), array('d', [0.996036916588859, 0.9705375301440049, 1.0031097750756206, 0.9968902249243793, 0.9907539227926027, 0.9527136141507776]), array('d', [1.0297487030451902, 1.0067777358312895, 1.0069345612991956, 0.9930654387008047, 0.9863170100650357, 0.9881408445393073]), array('d', [1.041076948272917, 1.0462418085057528, 1.0381294986970875, 0.9618705013029129, 0.8973213457492779, 0.8957283727216508]), array('d', [1.0502564175760933, 1.0507487739462145, 1.0044475657397713, 0.9955524342602287, 0.9630758767652495, 0.9625205533940567]), array('d', [0.9993449940289938, 0.9626920814103648, 0.9865471258491874, 1.0134528741508126, 1.0681990878573913, 1.0935694880834979]), array('d', [0.9536602964970431, 0.9693269988171206, 0.9772804337115

In [21]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "sigma [rad]", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, sigma_dynamic_dphi[ifile][ilayer],dphi_err,sigma_dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_sigma_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma_phi[ifile][ilayer][iphi][-1].Fit("pol2","Q","",)
            g_sigma_phi[ifile][ilayer][iphi][-1].GetFunction("pol2").SetLineColor(colors[ilayer%4])
            g_sigma_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/sigma_phi_{ifile}{iOption0}{is_sigmalised}.png")

In [22]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_layers}][3]="+"\n{")
for fit_functions_ifile in fit_functions:
    print("    {")
    for fits in fit_functions_ifile:
        print("        {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("    },")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][3]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(0),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(1),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("    },")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][2]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_mean_phi[ifile][ilayer][0][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][ilayer][0][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("    },")
print("};")



const float phi_sigma_pt_params[9][16][3]=
{
    {
        {0.001588, 0.006524, -4.347625},
        {0.001884, 0.005848, -4.499999},
        {0.001592, 0.006573, -3.833634},
        {0.001842, 0.004737, -3.561936},
        {0.001577, 0.009578, -2.583634},
        {0.001699, 0.00919, -2.601974},
        {0.001452, 0.008386, -2.503443},
        {0.001699, 0.01, -2.821974},
        {0.002, 0.01, -2.5},
        {0.002, 0.01, -2.5},
        {0.001992, 0.01, -2.5},
        {0.002, 0.01, -2.5},
        {0.001382, 0.0036, -2.512724},
        {0.001278, 0.004432, -2.574873},
        {0.00122, 0.004719, -3.098246},
        {0.00135, 0.005369, -3.289561},
    },
    {
        {0.001598, 0.007122, -4.5},
        {0.001752, 0.007321, -4.5},
        {0.00161, 0.006418, -3.932282},
        {0.001818, 0.006423, -3.942296},
        {0.001634, 0.01, -2.776529},
        {0.001674, 0.009554, -2.641094},
        {0.001492, 0.009296, -2.693943},
        {0.001701, 0.009817, -2.834362},
        {0.001985, 0.

In [23]:
outrootfile = root.TFile(f"output/dynamic_delta_fits/dynamicdeltasigma{iOption0}{sim}.root","RECREATE")
for g_sigma_f in g_sigma:
    for g_sigma_layer in g_sigma_f:
        for g_sigma_phi in g_sigma_layer:
            g_sigma_phi.Write()